# Modelo de Vasicek

Vamos a trabajar primero con parámetros inventados. Haremos una simulación. Luego usando la fórmula para el bono cupón cero estudiaremos el ajuste del modelo a un set de datos reales.

In [1]:
# Damos de alta los parámetros del modelo
gamma = 1
sigma = .01
r_ = .03
r0 = .009 # tasa corta a día de hoy

Veamos que forma tienen sus trayectorias

In [2]:
import numpy as np
import math
def vasicek_path(r0, gamma, r_, sigma):
    dt = 1 / 264.0
    sqdt = math.sqrt(dt)
    time = [0]
    sim = [r0]
    # Vamos a simular una trayectoria de 1 año
    r = r0
    for i in range(1, 264):
        r = r + gamma*(r_ - r) * dt + sigma * sqdt * np.random.normal()
        sim.append(r)
        time.append(i * dt)
    return time, sim

In [3]:
# Esta es una librería para visualizaciones
from bokeh.plotting import figure, show, output_notebook

def plot_simulation(r0, gamma, r_, sigma):
    sim = vasicek_path(r0, gamma, r_, sigma)

    # se define la data
    x = sim[0]
    y = sim[1]

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title="Trayectoria Vasicek", x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(x, y, legend="Vasicek", line_width=2)

    # show the results
    show(p)
    tasa_minima = round(min(y) * 100,2)
    tasa_maxima = round(max(y) * 100,2)
    rango = round(tasa_maxima - tasa_minima, 2)
    print("Tasa minima = " + str(tasa_minima))
    print("Tasa maxima = " + str(tasa_maxima))
    print("Rango = " + str(rango))

In [4]:
plot_simulation(r0, gamma, r_, sigma)

Loading BokehJS ...

Tasa minima = 0.84
Tasa maxima = 2.52
Rango = 1.68


In [5]:
# Recuperemos valores de factores de descuento
import csv
def get_curva(path_to_file):
    plazos = []
    tasas = []
    with open(path_to_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            plazos.append(row[0])
            tasas.append(row[1])
    plazos = plazos[1:]
    tasas = tasas[1:]
    plazos1 = []
    tasas1 = []
    for plazo in plazos:
        plazos1.append(float(plazo))
    for tasa in tasas:
        tasas1.append(float(tasa))
    # print(plazos1)
    # print(tasas1)
    return plazos1, tasas1

In [7]:
curva = get_curva("./curva_3.csv")
for x in zip(curva[0], curva[1]):
    print(x)

(0.002739726, 0.999975001)
(0.019178082, 0.999823573)
(0.038356164, 0.999647169)
(0.060273973, 0.999450913)
(0.087671233, 0.999201527)
(0.167123288, 0.998473099)
(0.252054795, 0.997633801)
(0.336986301, 0.996740906)
(0.419178082, 0.99584806)
(0.509589041, 0.994859891)
(0.75890411, 0.991800113)
(1.0, 0.988563827)
(1.506849315, 0.981183423)
(2.005479452, 0.972693527)
(3.002739726, 0.954199287)
(4.002739726, 0.934493447)
(5.002739726, 0.914165264)
(7.010958904, 0.851735228)
(10.00547945, 0.781818476)
(12.00821918, 0.736803541)
(15.0109589, 0.674157693)
(20.01369863, 0.582462093)
(25.01917808, 0.506693422)
(30.02465753, 0.442322476)
(40.02739726, 0.341431864)
(50.03287671, 0.267091985)


In [8]:
"""
Se definen las funciones asociadas a la formula para obtener un factor de descuento
con el modelo de Vasicek.
"""
def b_vasicek(r, r_, gamma, t):
    return 1 / gamma * (1 - math.exp(-gamma * t))

def vasicek_zero(r, r_, gamma, sigma, t):
    b = b_vasicek(r, r_, gamma, t)
    sigma2 = sigma ** 2
    a = (b - t) * (r_ - sigma2 / (2 * gamma ** 2)) - (sigma2 * b ** 2) / (4.0 * gamma)
    return math.exp(a - b * r)

In [9]:
# Probar la función
df = vasicek_zero(r0, r_, gamma, sigma, 1)
print(df)

0.9834218910243747


In [10]:
# se define la función objetivo para encontrar los parámetros
# x = [r_, gamma, sigma] valor inicial
def objective(x):
    error = 0.0
    for y in zip(curva[0], curva[1]):
        error += (y[1] - vasicek_zero(r0, x[0], x[1], x[2], y[0])) ** 2
    return error

In [11]:
# Usando scipy se encuentran los parámetros
import scipy.optimize as opt
x0 = [.03, 1, .01]
result = opt.minimize(objective, x0)
print(result)
optimo = result.x

      fun: 0.0007961056578792739
 hess_inv: array([[ 1.38800329e-03, -1.81797328e-01,  2.40435153e-02],
       [-1.81797328e-01,  3.74056465e+01, -2.65295135e-01],
       [ 2.40435153e-02, -2.65295135e-01,  1.91200294e+01]])
      jac: array([8.11523205e-06, 4.16985131e-08, 7.21374818e-07])
  message: 'Optimization terminated successfully.'
     nfev: 170
      nit: 28
     njev: 34
   status: 0
  success: True
        x: array([2.83881509e-02, 4.55038856e-01, 1.53974089e-05])


In [12]:
# Comparemos la curva real con la aproximacion
v_curva = []
for plazo in curva[0]:
    v_curva.append(vasicek_zero(r0, optimo[0], optimo[1], optimo[2], plazo))

# create a new plot with a title and axis labels
p1 = figure(title="Curvas de factores de descuento", x_axis_label='Tiempo', y_axis_label='df')

x = curva[0]
# add a line renderer with legend and line thickness
p1.line(x, curva[1], legend="Mkt curve", line_width=2, line_color="red")
p1.line(x, v_curva, legend="Model curve", line_width=2, line_color="blue")


# show the results
show(p1)

In [13]:
# Hagamos la misma comparación en tasa
def plot_mkt_rate(curva, modelo):
    mkt_rate = []
    for df in zip(curva[0], curva[1]):
        rate = - math.log(df[1]) / df[0]
        mkt_rate.append(rate)

    model_rate = []
    for df in zip(curva[0], modelo):
        rate = - math.log(df[1]) / df[0]
        model_rate.append(rate)

    # create a new plot with a title and axis labels
    p = figure(title="Curvas de Tasas Cero", x_axis_label='Tiempo', y_axis_label='Tasa')

    x = curva[0]
    # add a line renderer with legend and line thickness
    p.line(x, mkt_rate, legend="Mkt curve", line_width=2, line_color="red")
    p.line(x, model_rate, legend="Model curve", line_width=2, line_color="blue")


    # show the results
    show(p)
plot_mkt_rate(curva, v_curva)

In [14]:
# se define una nueva función objetivo para encontrar los parámetros
def objective1(x):
    error = 0.0
    for y in zip(curva[0], curva[1]):
        error += (y[1] - vasicek_zero(r0, x[0], x[1], sigma, y[0])) ** 2
    return error

x0 = [.03, 1]
result1 = opt.minimize(objective1, x0)
optimo1 = result1.x
print(optimo1)

[0.02863179 0.45223923]


In [15]:
# Veamos como queda la simulación
plot_simulation(r0, optimo1[0], optimo1[1], sigma)

Loading BokehJS ...

Tasa minima = 0.66
Tasa maxima = 2.69
Rango = 2.03


In [16]:
# ¿Y la curva?
# Comparemos la curva real con la aproximacion
v_curva1 = []
for plazo in curva[0]:
    v_curva1.append(vasicek_zero(r0, optimo1[0], optimo1[1], sigma, plazo))
    
plot_mkt_rate(curva, v_curva1)